CER030 - Sign Management Proxy certificate with generated CA
============================================================

This notebook signs the certificate created using:

-   [CER020 - Create Management Proxy
    certificate](../cert-management/cer020-create-management-service-proxy-cert.ipynb)

with the Big Data Cluster CA or the generated Root CA Certificate,
created using either:

-   [CER001 - Generate a Root CA
    certificate](../cert-management/cer001-create-root-ca.ipynb)
-   [CER003 - Upload existing Root CA
    certificate](../cert-management/cer003-upload-existing-root-ca.ipynb)

Steps
-----

### Parameters

In [ ]:
import getpass

app_name = "mgmtproxy"
scaledset_name = "mgmtproxy"
container_name = "service-proxy"
prefix_keyfile_name = "service-proxy"
common_name = "mgmtproxy-svc"

country_name = "US"
state_or_province_name = "Illinois"
locality_name = "Chicago"
organization_name = "Contoso"
organizational_unit_name = "Finance"
email_address = f"{getpass.getuser().lower()}@contoso.com"

ssl_configuration_file = "ca.openssl.cnf"

days = "398" # the number of days to certify the certificate for

certificate_filename = "cacert.pem"
private_key_filename = "cakey.pem"
use_bdc_ca = False

test_cert_store_root = "/var/opt/secrets/test-certificates"

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

def run(cmd, return_output=False, no_output=False, retry_count=0, base64_decode=False, return_as_json=False, regex_mask=None):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportability, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    # Display an install HINT, so the user can click on a SOP to install the missing binary
    #
    if which_binary == None:
        print(f"The path used to search for '{cmd_actual[0]}' was:")
        print(sys.path)

        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    cmd_display = cmd
    if regex_mask is not None:
        regex = re.compile(regex_mask)
        cmd_display = re.sub(regex, '******', cmd)
        
    print(f"START: {cmd_display} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    if base64_decode:
                                        import base64
                                        return base64.b64decode(output).decode('utf-8')
                                    else:
                                        return output

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        if base64_decode:
            import base64
            return base64.b64decode(output).decode('utf-8')
        else:
            return output



# Hints for tool retry (on transient fault), known errors and install guide
#
retry_hints = {'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use', 'Login timeout expired (0) (SQLDriverConnect)', 'SSPI Provider: No Kerberos credentials available',  ], 'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',  ], 'python': [ ], }
error_hints = {'azdata': [['Please run \'azdata login\' to first authenticate', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Can\'t open lib \'ODBC Driver 17 for SQL Server', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb'], ['NameError: name \'azdata_login_secret_name\' is not defined', 'SOP013 - Create secret for azdata login (inside cluster)', '../common/sop013-create-secret-for-azdata-login.ipynb'], ['ERROR: No credentials were supplied, or the credentials were unavailable or inaccessible.', 'TSG124 - \'No credentials were supplied\' error from azdata login', '../repair/tsg124-no-credentials-were-supplied.ipynb'], ['Please accept the license terms to use this product through', 'TSG126 - azdata fails with \'accept the license terms to use this product\'', '../repair/tsg126-accept-license-terms.ipynb'],  ], 'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb'],  ], 'python': [['Library not loaded: /usr/local/opt/unixodbc', 'SOP012 - Install unixodbc for Mac', '../install/sop012-brew-install-odbc-for-sql-server.ipynb'], ['WARNING: You are using pip version', 'SOP040 - Upgrade pip in ADS Python sandbox', '../install/sop040-upgrade-pip.ipynb'],  ], }
install_hint = {'azdata': [ 'SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb' ],  'kubectl': [ 'SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb' ],  }


print('Common functions defined successfully.')

### Get the Kubernetes namespace for the big data cluster

Get the namespace of the Big Data Cluster use the kubectl command line
interface .

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = run(f'kubectl get namespace --selector=MSSQL_CLUSTER -o jsonpath={{.items[0].metadata.name}}', return_output=True)
    except:
        from IPython.display import Markdown
        print(f"ERROR: Unable to find a Kubernetes namespace with label 'MSSQL_CLUSTER'.  SQL Server Big Data Cluster Kubernetes namespaces contain the label 'MSSQL_CLUSTER'.")
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print(f'The SQL Server Big Data Cluster Kubernetes namespace is: {namespace}')

### Create a temporary directory to stage files

In [ ]:
# Create a temporary directory to hold configuration files

import tempfile

temp_dir = tempfile.mkdtemp()

print(f"Temporary directory created: {temp_dir}")

### Helper function to save configuration files to disk

In [ ]:
# Define helper function 'save_file' to save configuration files to the temporary directory created above
import os
import io

def save_file(filename, contents):
    with io.open(os.path.join(temp_dir, filename), "w", encoding='utf8', newline='\n') as text_file:
      text_file.write(contents)

      print("File saved: " + os.path.join(temp_dir, filename))

print("Function `save_file` defined successfully.")

### Get name of the ‘Running’ `controller` `pod`

In [ ]:
# Place the name  of the 'Running' controller pod in variable `controller`

controller = run(f'kubectl get pod --selector=app=controller -n {namespace} -o jsonpath={{.items[0].metadata.name}} --field-selector=status.phase=Running', return_output=True)

print(f"Controller pod name: {controller}")

### Create Signing Request configuration file

In [ ]:
certificate = f"""
[ ca ]
default_ca    = CA_default      # The default ca section

[ CA_default ]
default_days     = 1000         # How long to certify for
default_crl_days = 30           # How long before next CRL
default_md       = sha256       # Use public key default MD
preserve         = no           # Keep passed DN ordering

x509_extensions = ca_extensions # The extensions to add to the cert

email_in_dn     = no            # Don't concat the email in the DN
copy_extensions = copy          # Required to copy SANs from CSR to cert

base_dir      = {test_cert_store_root}
certificate   = $base_dir/{certificate_filename}   # The CA certifcate
private_key   = $base_dir/{private_key_filename}    # The CA private key
new_certs_dir = $base_dir              # Location for new certs after signing
database      = $base_dir/index.txt    # Database index file
serial        = $base_dir/serial.txt   # The current serial number

unique_subject = no  # Set to 'no' to allow creation of
                     # several certificates with same subject.

[ req ]
default_bits       = 2048
default_keyfile    = {test_cert_store_root}/{private_key_filename}
distinguished_name = ca_distinguished_name
x509_extensions    = ca_extensions
string_mask        = utf8only

[ ca_distinguished_name ]
countryName         = Country Name (2 letter code)
countryName_default = {country_name}

stateOrProvinceName         = State or Province Name (full name)
stateOrProvinceName_default = {state_or_province_name}

localityName                = Locality Name (eg, city)
localityName_default        = {locality_name}

organizationName            = Organization Name (eg, company)
organizationName_default    = {organization_name}

organizationalUnitName         = Organizational Unit (eg, division)
organizationalUnitName_default = {organizational_unit_name}

commonName         = Common Name (e.g. server FQDN or YOUR name)
commonName_default = {common_name}

emailAddress         = Email Address
emailAddress_default = {email_address}


[ ca_extensions ]
subjectKeyIdentifier   = hash
authorityKeyIdentifier = keyid:always, issuer
basicConstraints       = critical, CA:true
keyUsage               = keyCertSign, cRLSign

[ signing_policy ]
countryName            = optional
stateOrProvinceName    = optional
localityName           = optional
organizationName       = optional
organizationalUnitName = optional
commonName             = supplied
emailAddress           = optional

[ signing_req ]
subjectKeyIdentifier   = hash
authorityKeyIdentifier = keyid,issuer
basicConstraints       = CA:FALSE
keyUsage               = digitalSignature, keyEncipherment
"""

save_file(ssl_configuration_file, certificate)


### Copy certificate configuration to `controller` `pod`

In [ ]:
import os

cwd = os.getcwd()
os.chdir(temp_dir) # Use chdir to workaround kubectl bug on Windows, which incorrectly processes 'c:\' on kubectl cp cmd line 

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "mkdir -p {test_cert_store_root}/{app_name}"')

run(f'kubectl cp {ssl_configuration_file} {controller}:{test_cert_store_root}/{app_name}/{ssl_configuration_file} -c controller -n {namespace}')

os.chdir(cwd)

### Obtain CA certificate and extract the private key from pfx if Big Data Cluster CA is used

In [ ]:
if use_bdc_ca:

  cmd = f"cp /var/run/secrets/certificates/rootca/cluster-ca-certificate.crt {test_cert_store_root}/{certificate_filename}"

  run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

  cmd = f"NO_PASSWORD= openssl pkcs12 -in /var/run/secrets/certificates/rootca/cluster-ca-certificate.p12 -out {test_cert_store_root}/{private_key_filename} -nocerts -nodes -password env:NO_PASSWORD"

  run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Set next serial number

In [ ]:
run(f'kubectl exec {controller} -n {namespace} -c controller -- bash -c "test -f {test_cert_store_root}/index.txt || touch {test_cert_store_root}/index.txt"')
run(f"""kubectl exec {controller} -n {namespace} -c controller -- bash -c "test -f {test_cert_store_root}/serial.txt || echo '00' > {test_cert_store_root}/serial.txt" """)

current_serial_number = run(f"""kubectl exec {controller} -n {namespace} -c controller -- bash -c "cat {test_cert_store_root}/serial.txt" """, return_output=True)

# The serial number is hex
new_serial_number = int(f"0x{current_serial_number}", 0) + 1

run(f"""kubectl exec {controller} -n {namespace} -c controller -- bash -c "echo '{new_serial_number:02X}' > {test_cert_store_root}/serial.txt" """)

### Create private key and certificate signing request

Use openssl ca to create a private key and signing request. See:

-   https://www.openssl.org/docs/man1.0.2/man1/ca.html

In [ ]:
cmd = f"openssl ca -notext -batch -config {test_cert_store_root}/{app_name}/ca.openssl.cnf -policy signing_policy -extensions signing_req -out {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-certificate.pem -infiles {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-signingrequest.csr"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Display certificate

Use openssl x509 to display the certificate, so it can be visually
verified to be correct.

In [ ]:
cmd = f"openssl x509 -nameopt compat -in {test_cert_store_root}/{app_name}/{prefix_keyfile_name}-certificate.pem -text -noout"

run(f'kubectl exec {controller} -c controller -n {namespace} -- bash -c "{cmd}"')

### Clean up temporary directory for staging configuration files

In [ ]:
# Delete the temporary directory used to hold configuration files

import shutil

shutil.rmtree(temp_dir)

print(f'Temporary directory deleted: {temp_dir}')

In [ ]:
print("Notebook execution is complete.")

Related
-------

- [CER031 - Sign Knox certificate with generated CA](../cert-management/cer031-sign-knox-generated-cert.ipynb)
- [CER020 - Create Management Proxy certificate](../cert-management/cer020-create-management-service-proxy-cert.ipynb)
- [CER040 - Install signed Management Proxy certificate](../cert-management/cer040-install-service-proxy-cert.ipynb)
